![](./Resources/ai-sk-add-wechat.png)

# SK  使用依赖注入

[依赖注入](https://learn.microsoft.com/en-us/aspnet/core/fundamentals/dependency-injection)使得应用程序关注点分离，易于构建松耦合和易于测试的应用，SK 很好的支持依赖注入。本节将详细介绍如何更好的使用依赖注 入进行SK相关服务的注册和管理。

## 注册 Kernel 服务

### 导入相关NuGet包和定义的类

In [14]:
#r "nuget:Microsoft.Extensions.Configuration"
#r "nuget:Microsoft.Extensions.Configuration.Json"
#r "nuget:Microsoft.Extensions.Configuration.Binder"
#r "nuget:Microsoft.SemanticKernel"
#r "nuget:Microsoft.SemanticKernel.Connectors.OpenAI"
#r "nuget:Microsoft.SemanticKernel.Connectors.AzureOpenAI"
// 安装依赖注入NuGet包
#r "nuget:Microsoft.Extensions.DependencyInjection"


#!import Config/AiProvider.cs
#!import Config/AiOptions.cs
#!import Config/AiSettings.cs

Installed Packages Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Binder, 8.0.2 Microsoft.Extensions.Configuration.Json, 8.0.1 Microsoft.Extensions.DependencyInjection, 8.0.1 Microsoft.SemanticKernel, 1.25.0 Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.25.0 Microsoft.SemanticKernel.Connectors.OpenAI, 1.25.0

### 抽象AI 服务注册器

对于SK 而言，其核心就是基于构建好的`Kernel`进行服务调用，因此AI 服务注册器的核心就是构建`Kernel`。就目前而言大多数AI 服务提供商均提供文本补全和嵌入接口，因此定义了两个抽象方法供子类自行实现：
* `RegisterChatCompletionService`： 用于注册聊天补全服务
* `RegisterEmbeddingService`：用于注册文本嵌入服务

其中包含两个虚方法：
* `Register`，其核心就是使用`AddKeyedTransient`按照AI 提供商的Code进行注册，注册为不同的命名Kernel，以便运行时动态获取。具体使用到了[Keyed Services](https://learn.microsoft.com/en-us/aspnet/core/fundamentals/dependency-injection) 特性。
* `BuildKernel`，其核心就是构建`Kernel`，主要是对`RegisterChatCompletionService`和`RegisterEmbeddingService` 两个方法的调用。当需要注册其他服务时，可以自行重载。

In [15]:
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;

public abstract class AiProviderRegister
{
    public abstract AiProviderType AiProviderType { get; }

    public virtual void Register(IServiceCollection services, AiProvider aiProvider)
    {
        // 为指定AiProvider 注册专用Kernel服务
        services.AddKeyedTransient<Kernel>(aiProvider.Code, (sp, key) => BuildKernel(sp, aiProvider));
    }

    public virtual Kernel BuildKernel(IServiceProvider serviceProvider, AiProvider aiProvider)
    {
        // 创建Kernel构建器
        var builder = Kernel.CreateBuilder();

        RegisterChatCompletionService(builder, serviceProvider, aiProvider);
        RegisterEmbeddingService(builder, serviceProvider, aiProvider);
        
        // Register other services if needed

        return builder.Build();
    }

    protected abstract void RegisterChatCompletionService(IKernelBuilder builder, IServiceProvider provider, AiProvider aiProvider);

    protected abstract void RegisterEmbeddingService(IKernelBuilder builder, IServiceProvider provider, AiProvider aiProvider);
}

### 实现AI 服务注册器
接下来就为具体的AI服务提供商类型分别实现自定义的AI 服务注册器：

#### OpenAI 服务注册器

In [16]:
using System.Diagnostics.CodeAnalysis;
using Microsoft.SemanticKernel;

public class OpenAiRegister : AiProviderRegister
{
    public override AiProviderType AiProviderType => AiProviderType.OpenAI;

    protected override void RegisterChatCompletionService(IKernelBuilder builder, IServiceProvider provider, AiProvider aiProvider)
    {
        var chatModelId = aiProvider.GetChatCompletionApiService()?.ModelId;
        if (string.IsNullOrWhiteSpace(chatModelId))
        {
            return;
        }

        builder.AddOpenAIChatCompletion(modelId: chatModelId, apiKey: aiProvider.ApiKey);
    }

    [Experimental("SKEXP0010")]
    protected override void RegisterEmbeddingService(IKernelBuilder builder, IServiceProvider provider, AiProvider aiProvider)
    {
        var embeddingModelId = aiProvider.GetEmbeddingApiService()?.ModelId;
        if (string.IsNullOrWhiteSpace(embeddingModelId))
        {
            return;
        }

        builder.AddOpenAITextEmbeddingGeneration(embeddingModelId, aiProvider.ApiKey);
    }
}

#### AzureOpenAI 服务注册器

In [4]:
using System.Diagnostics.CodeAnalysis;
using Microsoft.SemanticKernel;

public class AzureOpenAiRegister : AiProviderRegister
{
    public override AiProviderType AiProviderType => AiProviderType.AzureOpenAI;

    protected override void RegisterChatCompletionService(IKernelBuilder builder, IServiceProvider provider, AiProvider aiProvider)
    {
        var modelId = aiProvider.GetChatCompletionApiService()?.ModelId;

        // 如果未找到ModelId，说明该AI提供商未提供对应服务，则直接返回builder
        if (string.IsNullOrWhiteSpace(modelId))
        {
            return;
        }

        builder.AddAzureOpenAIChatCompletion(
            deploymentName: modelId,
            endpoint: aiProvider.ApiEndpoint,
            apiKey: aiProvider.ApiKey);
    }

    [Experimental("SKEXP0010")]
    protected override void RegisterEmbeddingService(IKernelBuilder builder, IServiceProvider provider, AiProvider aiProvider)
    {
        var modelId = aiProvider.GetEmbeddingApiService()?.ModelId;

        // 如果未找到ModelId，说明该AI提供商未提供对应服务，则直接返回builder
        if (string.IsNullOrWhiteSpace(modelId))
        {
            return;
        }

        builder.AddAzureOpenAITextEmbeddingGeneration(
            deploymentName: modelId,
            endpoint: aiProvider.ApiEndpoint,
            apiKey: aiProvider.ApiKey);
    }
}

#### 兼容OpenAI 的服务注册器

In [5]:
using System.ClientModel;
using System.Diagnostics.CodeAnalysis;
using Microsoft.SemanticKernel;
using OpenAI;

public class OpenAiCompatibleAiRegister : AiProviderRegister
{
    public override AiProviderType AiProviderType => AiProviderType.OpenAI_Compatible;

    protected override void RegisterChatCompletionService(IKernelBuilder builder, IServiceProvider provider, AiProvider aiProvider)
    {
        var chatModelId = aiProvider.GetChatCompletionApiService()?.ModelId;
        if (string.IsNullOrWhiteSpace(chatModelId))
        {
            return;
        }

        OpenAIClientOptions clientOptions = new OpenAIClientOptions
        {
            Endpoint = new Uri(aiProvider.ApiEndpoint)
        };

        OpenAIClient client = new(new ApiKeyCredential(aiProvider.ApiKey), clientOptions);

        builder.AddOpenAIChatCompletion(modelId: chatModelId, openAIClient: client);
    }

    [Experimental("SKEXP0010")]
    protected override void RegisterEmbeddingService(IKernelBuilder builder, IServiceProvider provider, AiProvider aiProvider)
    {
        var embeddingModelId = aiProvider.GetEmbeddingApiService()?.ModelId;
        if (string.IsNullOrWhiteSpace(embeddingModelId))
        {
            return;
        }

        OpenAIClientOptions clientOptions = new OpenAIClientOptions
        {
            Endpoint = new Uri(aiProvider.ApiEndpoint)
        };

        OpenAIClient client = new(new ApiKeyCredential(aiProvider.ApiKey), clientOptions);

        builder.AddOpenAITextEmbeddingGeneration(embeddingModelId, openAIClient: client);
    }
}

### 定义AI服务注册工厂类

有了不同类型的AI 服务注册类，即可通过简单定义一个简单的静态工厂来根据类型获取对应的AI服务注册类：

In [6]:
public static class AiProviderRegisterFactory
{
    public static AiProviderRegister Create(AiProviderType aiProviderType)
    {
        return aiProviderType switch
        {
            AiProviderType.OpenAI => new OpenAiRegister(),
            AiProviderType.OpenAI_Compatible => new OpenAiCompatibleAiRegister(),
            AiProviderType.AzureOpenAI => new AzureOpenAiRegister(),
            _ => throw new NotImplementedException($"No AI register for nameof(aiProviderType)")
        };
    }
}

### 定义服务注册扩展方法

In [8]:
using Microsoft.Extensions.DependencyInjection;

public static IServiceCollection RegisterKernels(this IServiceCollection services)
{
    // 从配置文件中加载AI配置
    var aiOptions = AiSettings.LoadAiProvidersFromFile();
    // 注册其他AI服务提供商
    foreach (var aiProvider in aiOptions.Providers)
    {
        var providerRegister = AiProviderRegisterFactory.Create(aiProvider!.AiType);
    
        providerRegister.Register(services, aiProvider);
    }
    return services;
}

### 批量注册服务

创建依赖注入容器并通过调用扩展方法完成服务的注册：

In [17]:
// 初始化依赖注入容器
var services = new ServiceCollection();

// 注册Kernel服务
services.RegisterKernels();

// 构建ServiceProvider
var serviceProvider = services.BuildServiceProvider();

## 使用Kernel 服务

上面完成了依赖注入容器的创建、服务的注册和依赖容器的构建，接下来就可以使用以下方式获取服务并调用：

In [19]:
using Microsoft.SemanticKernel.ChatCompletion;

// 获取Kernel服务
var kernel = serviceProvider.GetKeyedService<Kernel>("oneapi");
var response = await kernel.InvokePromptAsync("一句话简单介绍Semantic Kernel。");
response.Display();

var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();
var chatCompletionResponse = await chatCompletionService.GetChatMessageContentAsync("一句话简单介绍LangChain。");
chatCompletionResponse.Display();

Semantic Kernel是一个轻量级的开源框架，通过它可以快速使用不同编程语言(C#/Python/Java)结合LLMs(OpenAI、Azure OpenAI、Hugging Face等模型)构建智能应用，简化将人工智能(AI)集成到现有解决方案中的流程。 Function InvokePromptAsync_59c8cb8888f24f0fa0fa9991649d36c9 Name InvokePromptAsync_59c8cb8888f24f0fa0fa9991649d36c9 PluginName <null> Description Generic function, unknown purpose Metadata Microsoft.SemanticKernel.KernelFunctionMetadata Name InvokePromptAsync_59c8cb8888f24f0fa0fa9991649d36c9 PluginName <null> Description Generic function, unknown purpose Parameters (empty) ReturnParameter Microsoft.SemanticKernel.KernelReturnParameterMetadata Description ParameterType <null> Schema <null> AdditionalProperties (empty) ExecutionSettings (empty) Metadata key type value Id System.String chatcmpl-f4fc321fabf44bf2a0f0e63b0618b3cf CreatedAt System.DateTimeOffset 2024-10-31 01:59:45Z SystemFingerprint <null> <null> Usage OpenAI.Chat.ChatTokenUsage OpenAI.Chat.ChatTokenUsage OutputTokenCount 67 InputTokenCount 39 TotalTokenCount 106 OutputTokenDetails <null> Refusal <null> <null> FinishReason System.String Stop ContentTokenLogProbabilities OpenAI.ChangeTrackingList<OpenAI.Chat.ChatTokenLogProbabilityDetails> Culture Parent Parent Parent Parent Parent LCID 127 KeyboardLayoutId 127 Name IetfLanguageTag DisplayName Invariant Language (Invariant Country) NativeName Invariant Language (Invariant Country) EnglishName Invariant Language (Invariant Country) TwoLetterISOLanguageName iv ThreeLetterISOLanguageName ivl ThreeLetterWindowsLanguageName IVL CompareInfo CompareInfo - TextInfo TextInfo - IsNeutralCulture False CultureTypes SpecificCultures, InstalledWin32Cultures NumberFormat System.Globalization.NumberFormatInfo DateTimeFormat System.Globalization.DateTimeFormatInfo Calendar System.Globalization.GregorianCalendar OptionalCalendars [ System.Globalization.GregorianCalendar ] UseUserOverride False IsReadOnly True LCID 127 KeyboardLayoutId 127 Name IetfLanguageTag DisplayName Invariant Language (Invariant Country) NativeName Invariant Language (Invariant Country) EnglishName Invariant Language (Invariant Country) TwoLetterISOLanguageName iv ThreeLetterISOLanguageName ivl ThreeLetterWindowsLanguageName IVL CompareInfo CompareInfo - Name Version System.Globalization.SortVersion LCID 127 TextInfo TextInfo - ANSICodePage 1252 OEMCodePage 437 MacCodePage 10000 EBCDICCodePage 37 LCID 127 CultureName IsReadOnly True ListSeparator , IsRightToLeft False IsNeutralCulture False CultureTypes SpecificCultures, InstalledWin32Cultures NumberFormat System.Globalization.NumberFormatInfo CurrencyDecimalDigits 2 CurrencyDecimalSeparator . IsReadOnly True CurrencyGroupSizes [ 3 ] NumberGroupSizes [ 3 ] PercentGroupSizes [ 3 ] CurrencyGroupSeparator , CurrencySymbol ¤ NaNSymbol NaN CurrencyNegativePattern 0 NumberNegativePattern 1 PercentPositivePattern 0 PercentNegativePattern 0 NegativeInfinitySymbol -Infinity NegativeSign - NumberDecimalDigits 2 NumberDecimalSeparator . NumberGroupSeparator , CurrencyPositivePattern 0 PositiveInfinitySymbol Infinity PositiveSign + PercentDecimalDigits 2 PercentDecimalSeparator . PercentGroupSeparator , PercentSymbol % PerMilleSymbol ‰ NativeDigits [ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9 ] DigitSubstitution None DateTimeFormat System.Globalization.DateTimeFormatInfo AMDesignator AM Calendar System.Globalization.GregorianCalendar DateSeparator / FirstDayOfWeek Sunday CalendarWeekRule FirstDay FullDateTimePattern dddd, dd MMMM yyyy HH:mm:ss LongDatePattern dddd, dd MMMM yyyy LongTimePattern HH:mm:ss MonthDayPattern MMMM dd PMDesignator PM RFC1123Pattern ddd, dd MMM yyyy HH':'mm':'ss 'GMT' ShortDatePattern MM/dd/yyyy ShortTimePattern HH:mm SortableDateTimePattern yyyy'-'MM'-'dd'T'HH':'mm':'ss TimeSeparator : UniversalSortableDateTimePattern yyyy'-'MM'-'dd HH':'mm':'ss'Z' YearMonthPattern yyyy MMMM AbbreviatedDayNames [ Sun, Mon, Tue, Wed, Thu, Fri, Sat ] ShortestDayNames [ Su, Mo, Tu, We, Th, Fr, Sa ] DayNames [ Sunday, Monday, Tuesday, Wednesday, Thursday, Friday, Saturday ] AbbreviatedMonthNames [ Jan, 

LangChain是一个开源框架，旨在帮助开发人员使用语言模型构建端到端的应用程序。它提供了一套工具、组件和接口，可简化创建由大型语言模型 (LLM) 和聊天模型提供支持的应用程序的过程。LangChain 可以轻松管理与语言模型的交互，将多个组件链接在一起，并集成额外的资源，例如 API 和数据库。 ToolCalls [ ] IsUndefined True Count 0 IsReadOnly False (values) (empty) AuthorName <null> Role Assistant Label Assistant Content LangChain是一个开源框架，旨在帮助开发人员使用语言模型构建端到端的应用程序。它提供了一套工具、组件和接口，可简化创建由大型语言模型 (LLM) 和聊天模型提供支持的应用程序的过程。LangChain 可以轻松管理与语言模型的交互，将多个组件链接在一起，并集成额外的资源，例如 API 和数据库。 Items [ LangChain是一个开源框架，旨在帮助开发人员使用语言模型构建端到端的应用程序。它提供了一套工具、组件和接口，可简化创建由大型语言模型 (LLM) 和聊天模型提供支持的应用程序的过程。LangChain 可以轻松管理与语言模型的交互，将多个组件链接在一起，并集成额外的资源，例如 API 和数据库。 ] Count 1 (values) index value 0 LangChain是一个开源框架，旨在帮助开发人员使用语言模型构建端到端的应用程序。它提供了一套工具、组件和接口，可简化创建由大型语言模型 (LLM) 和聊天模型提供支持的应用程序的过程。LangChain 可以轻松管理与语言模型的交互，将多个组件链接在一起，并集成额外的资源，例如 API 和数据库。 Text LangChain是一个开源框架，旨在帮助开发人员使用语言模型构建端到端的应用程序。它提供了一套工具、组件和接口，可简化创建由大型语言模型 (LLM) 和聊天模型提供支持的应用程序的过程。LangChain 可以轻松管理与语言模型的交互，将多个组件链接在一起，并集成额外的资源，例如 API 和数据库。 Encoding System.Text.UTF8Encoding+UTF8EncodingSealed Preamble System.Text.UTF8Encoding+UTF8EncodingSealed BodyName utf-8 EncodingName Unicode (UTF-8) HeaderName utf-8 WebName utf-8 WindowsCodePage 1200 IsBrowserDisplay True IsBrowserSave True IsMailNewsDisplay True IsMailNewsSave True IsSingleByte False EncoderFallback System.Text.EncoderReplacementFallback DefaultString � MaxCharCount 1 DecoderFallback System.Text.DecoderReplacementFallback DefaultString � MaxCharCount 1 IsReadOnly True CodePage 65001 MimeType <null> InnerContent <null> ModelId <null> Metadata <null> Encoding System.Text.UTF8Encoding+UTF8EncodingSealed Preamble System.Text.UTF8Encoding+UTF8EncodingSealed BodyName utf-8 EncodingName Unicode (UTF-8) HeaderName utf-8 WebName utf-8 WindowsCodePage 1200 IsBrowserDisplay True IsBrowserSave True IsMailNewsDisplay True IsMailNewsSave True IsSingleByte False EncoderFallback System.Text.EncoderReplacementFallback DefaultString � MaxCharCount 1 DecoderFallback System.Text.DecoderReplacementFallback DefaultString � MaxCharCount 1 IsReadOnly True CodePage 65001 Source <null> MimeType <null> InnerContent OpenAI.Chat.ChatCompletion CreatedAt 2024-10-31 01:59:51Z FinishReason Stop ContentTokenLogProbabilities [ ] IsUndefined True Count 0 IsReadOnly False (values) (empty) RefusalTokenLogProbabilities [ ] IsUndefined True Count 0 IsReadOnly False (values) (empty) Role Assistant Content index value 0 OpenAI.Chat.ChatMessageContentPart Kind Text Text LangChain是一个开源框架，旨在帮助开发人员使用语言模型构建端到端的应用程序。它提供了一套工具、组件和接口，可简化创建由大型语言模型 (LLM) 和聊天模型提供支持的应用程序的过程。LangChain 可以轻松管理与语言模型的交互，将多个组件链接在一起，并集成额外的资源，例如 API 和数据库。 ImageUri <null> ImageBytes <null> ImageBytesMediaType <null> ImageDetailLevel <null> Refusal <null> ToolCalls [ ] IsUndefined True Count 0 IsReadOnly False (values) (empty) Refusal <null> FunctionCall <null> Id chatcmpl-35e6c1c8d4274161bcf82b0ca383294a Model <null> SystemFingerprint <null> Usage OpenAI.Chat.ChatTokenUsage OutputTokenCount 90 InputTokenCount 38 TotalTokenCount 128 OutputTokenDetails <null> ModelId lite Metadata key type value Id System.String chatcmpl-35e6c1c8d4274161bcf82b0ca383294a CreatedAt System.DateTimeOffset 2024-10-31 01:59:51Z SystemFingerprint <null> <null> Usage OpenAI.Chat.ChatTokenUsage OpenAI.Chat.ChatTokenUsage OutputTokenCount 90 InputTokenCount 38 TotalTokenCount 128 OutputTokenDetails <null> Refusal <null> <null> FinishReason System.String Stop ContentTokenLogProbabilities OpenAI.ChangeTrackingList<OpenAI.Chat.ChatTokenLogProbabilityDetails>

参考：
https://devblogs.microsoft.com/semantic-kernel/using-semantic-kernel-with-dependency-injection/

https://github.com/microsoft/semantic-kernel/blob/main/dotnet/samples/Concepts/DependencyInjection/HttpClient_Registration.cs